In [1]:
import collections
import copy
import datetime
import itertools
import sys

import json
import pathlib
import re
from pprint import pprint

import nest_asyncio
nest_asyncio.apply()
import texttable

import pytourney
import pyfumbbl
import cibblbibbl

In [2]:
show_Ids = True
G = cibblbibbl.CIBBL
Achievement = cibblbibbl.achievement.Achievement
Team = cibblbibbl.team.Team
player = cibblbibbl.player.player
Match = cibblbibbl.match.Match
Matchup = cibblbibbl.matchup.Matchup
Replay = cibblbibbl.replay.Replay
Tournament = cibblbibbl.tournament.handler.default.Tournament
NormalPlayer = cibblbibbl.player.NormalPlayer
StarPlayer = cibblbibbl.player.StarPlayer

In [3]:
G.init()

In [26]:
lastaliveplayers = {}
for Te in G.teams:
    lastT = Te.tournaments[-1]
    lastaliveplayers[Te] = lastT.lastaliveplayers(Te)

In [27]:
lastaliveplayers

{Team(930817): {NormalPlayer('12651553'),
  NormalPlayer('12651554'),
  NormalPlayer('12651555'),
  NormalPlayer('12651556'),
  NormalPlayer('12651559'),
  NormalPlayer('12651563'),
  NormalPlayer('12651564'),
  NormalPlayer('12651565'),
  NormalPlayer('12651569'),
  NormalPlayer('12698746')},
 Team(897538): {NormalPlayer('12124716'),
  NormalPlayer('12124717'),
  NormalPlayer('12124718'),
  NormalPlayer('12124719'),
  NormalPlayer('12124720'),
  NormalPlayer('12124721'),
  NormalPlayer('12124722'),
  NormalPlayer('12124723'),
  NormalPlayer('12124724'),
  NormalPlayer('12124725'),
  NormalPlayer('12124728'),
  NormalPlayer('12164624'),
  NormalPlayer('12164625')},
 Team(923140): {NormalPlayer('12523353'),
  NormalPlayer('12523354'),
  NormalPlayer('12523356'),
  NormalPlayer('12523358'),
  NormalPlayer('12523364'),
  NormalPlayer('12523367'),
  NormalPlayer('12536501'),
  NormalPlayer('12537431'),
  NormalPlayer('12597792'),
  NormalPlayer('12637501'),
  NormalPlayer('12637502'),
  No

In [30]:
d = collections.defaultdict(set)
for Te, Pls in lastaliveplayers.items():
    d[Te] |= Te.achievements
    for Pl in Pls:
        d[Te] |= Pl.achievements

In [55]:
v = {}
for Te, As in d.items():
    v2 = collections.defaultdict(lambda: 0)
    for A in As:
        if A.subject_typename == "Team":
            T = A.tournament
            while T.above:
                T = T.above
            v2[T] += A.prestige()
    prestige = sum(A.prestige() for A in As)
    ties = [v2[T] for T in sorted(v2, reverse=True) if T.friendly == "no"]
    while True:
        try:
            x = ties.pop()
        except IndexError:
            break
        if x:
            ties.append(x)
            break
    ties += [0] * (6 - len(ties))
    v[Te] = (prestige, tuple(ties))

In [56]:
for Te in sorted(v, key=v.get, reverse=True):
    print(f'[{Te.Id}] {Te.name} : {v[Te][0]} {v[Te][1]}')

[867862] Everlasting Vultures of Settra : 995 (160, 190, 210, 80, 60, 50)
[876774] Congregation of Holy Touchdown : 975 (200, 170, 130, 35, 110, 50)
[886114] Eye and Arrow : 930 (40, 150, 280, 100, 40, 80)
[854678] Lords of the Silent Death : 920 (140, 140, 260, 110, 25, 35)
[897376] Darkenwood Sentinels : 860 (60, 200, 170, 80, 120, 70, 40)
[895929] Fangsnapper's Clawpack : 845 (50, 160, 230, 105, 55, 60)
[878945] Eternal Ecstasy : 840 (110, 170, 210, 30, 80, 50)
[874545] Stone Prince's Men : 805 (150, 190, 180, 75, 50, 100)
[854257] Grand Celestial Delegate : 800 (110, 170, 170, 110, 45, 50)
[871787] Malignant Curators : 785 (50, 220, 140, 40, 80, 0)
[903654] Scions of Neferata : 770 (80, 170, 190, 115, 5, 50)
[892248] Crimson Fins : 765 (50, 170, 230, 75, 55, 15, 35)
[874452] Way of the Dragon : 755 (60, 140, 50, 50, 100, 130)
[888994] Beerbog Behemoths : 720 (110, 110, 120, 15, 70, 40)
[854600] Montetriste Noblemen : 710 (60, 120, 50, 55, 70, 125)
[886002] Doom Glade Demons : 690 (

In [36]:
As = d[Team(867862)]

In [39]:
for A in sorted(As, key=lambda A: (A.tournament, A)):
    print(A)
    print(A.tournament, A.subject, A.prestige())
    print()

Match Result Prestige: Everlasting Vultures of Settra (110 Prestige Points)
Tournament('cibbl', '42425') Everlasting Vultures of Settra 0

Standings Prestige:  1. Everlasting Vultures of Settra (60 Prestige Points)
Tournament('cibbl', '42425') Everlasting Vultures of Settra 0

Winter's Glory: Everlasting Vultures of Settra
Tournament('cibbl', '42805') Everlasting Vultures of Settra 0

Match Result Prestige: Everlasting Vultures of Settra (70 Prestige Points)
Tournament('cibbl', '42890') Everlasting Vultures of Settra 0

Standings Prestige:  1. Everlasting Vultures of Settra (70 Prestige Points)
Tournament('cibbl', '42890') Everlasting Vultures of Settra 0

Match Result Prestige: Everlasting Vultures of Settra (60 Prestige Points)
Tournament('cibbl', '43154') Everlasting Vultures of Settra 0

Match Result Prestige: Everlasting Vultures of Settra (90 Prestige Points)
Tournament('cibbl', '43566') Everlasting Vultures of Settra 0

Standings Prestige:  4. Everlasting Vultures of Settra (20 

In [40]:
T = G.tournaments["44753"]

In [42]:
print(T.name)

Y3, Autumn - Gold Division


In [ ]:
T = sorted(Te.tournaments)[-1]
T

In [21]:
T.lastaliveplayers(Te)

{NormalPlayer('12416612'),
 NormalPlayer('12416620'),
 NormalPlayer('12416633'),
 NormalPlayer('12416640'),
 NormalPlayer('12416642'),
 NormalPlayer('12416645'),
 NormalPlayer('12416646'),
 NormalPlayer('12416647'),
 NormalPlayer('12416649'),
 NormalPlayer('12416651'),
 NormalPlayer('12423760'),
 NormalPlayer('12498397'),
 NormalPlayer('12498398')}